# Sanity check of ARAPCK results: comparison with QCFFPI results on the same 10nm $\times$ 10nm MAC structures

In [2]:
import numpy as np
from qcnico.qcffpi_io import read_energies, read_MO_file
from utils_arpack import gen_mos, gen_energies, find_redundant_MOs


def gen_qcffpi_energies(edir,lbls, natoms,filename_template='orb_energy_pCNN_10x10'):
    for n, N in zip(lbls,natoms):
        yield read_energies(edir + f'{filename_template}-{n}.dat', Natoms=N)

def gen_pos_MOs(Mdir,lbls, natoms, filename_template='MOs_pCNN_10x10'):
    for n, N in zip(lbls,natoms):
        yield read_MO_file(Mdir + f'{filename_template}-{n}.dat', Natoms=N)


qcffpi_dir = '/Users/nico/Desktop/simulation_outputs/qcffpi_data/'
qcffpi_edir = qcffpi_dir + 'orbital_energies/10x10_pCNN_MAC_ensemble_TB/'
qcffpi_Mdir = qcffpi_dir + 'MO_coefs/10x10_pCNN_MAC_ensemble_TB/'

arpack_dir = '/Users/nico/Desktop/simulation_outputs/percolation/10x10/'
arpack_edir = arpack_dir + 'eARPACK/'
arpack_Mdir = arpack_dir + 'MOs_ARPACK/'

natoms_file = qcffpi_dir + 'natoms.txt'

with open(natoms_file) as fo:
    fo.readline()
    fo.readline()
    lines = fo.readlines()

lbls_natoms = np.array([list(map(int, l.strip().split())) for l in lines])

lbls, natoms = lbls_natoms.T

energies_qcffpi = gen_qcffpi_energies(qcffpi_edir, lbls, natoms)
MOs_qcffpi = gen_pos_MOs(qcffpi_Mdir, lbls, natoms)

energies_arpack = gen_energies(arpack_edir,lbls)
MOs_arpack = gen_mos(arpack_Mdir,lbls)

For each MAC sample, find the proportion of ARPACK eigenpairs that are basically equivalent to QCFFPI eigenpairs (with similiarity threshold $\epsilon = 10^{-6}$ )

In [3]:
equiv_fraction = np.zeros(len(lbls))

for k, eq, ea, Mq, Ma in zip(range(len(lbls)), energies_qcffpi,energies_arpack,MOs_qcffpi, MOs_arpack):
    ii, jj = find_redundant_MOs(ea, eq, Mq, Ma, e_eps=1e-6, M_eps=1e-6)
    equiv_fraction[k] = len(ii)/ea.shape[0]
        

FileNotFoundError: [Errno 2] No such file or directory: '/Users/nico/Desktop/simulation_outputs/percolation/10x10/eARPACK//eARPACK_bigMAC-14.npy'